In [11]:
###Run this code to create a JSON file that diables auto-quotes and auto-brackets. After executing the Python 
#command, or manually creating the file, restart your Jupyter notebook, and it should stop auto-closing quotes
#and brackets.

#from notebook.services.config import ConfigManager
#c = ConfigManager()
#c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

###Keyboard Shortcuts
#Ctrl + Enter: Run single cell of code (similar to R)
#Ctrl + Shift + Enter: Run entire notebook (similar to R)

In [18]:
#Intro to Web Scraping and BeautifulSoup
#https://www.youtube.com/watch?v=XQgXKtPSzUI

#exporting to excel
#df.to_excel("excel_file_name.xlsx")

import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
#import re #for splitting strings using multiple delimiters
import time
import random

names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

In [19]:
#When populating species_XXX variables, make sure that they are all lowercase; to make comparison easier
#Common name on left side, taxonomy on right
species_vft = ['venus','flytrap','flytraps','dionaea','muscipula']
species_neps = ['tropical','asian','nepenthes']
species_sarracenia = ['american','sarracenia']
species_sundew = ['sundew','sundews','drosera']
species_utric = ['bladderwort','bladderworts','utricularia']
species_pings = ['butterwort','butterworts','pinguicula']
species_cephs = ['australian','cephalotus']
species_darlingtonia = ['cobra','lily','lilies','darlingtonia']
species_heli = ['sun','heliamphora']
#species_special = ['specimen plants','carnivero exclusives']

species_all = species_vft + species_neps + species_sarracenia + species_sundew + species_utric \
            + species_pings + species_cephs + species_darlingtonia + species_heli
    
#function to check type of species
#name is the name of the item as a string, returns a string of the type of species
def check_species(name):
    #cleaning: replacing special characters with spaces, all lowercase then splitting words into list
    kind_list = name.strip().replace('(','').replace(')','').replace('[','').replace(']','').replace(',','')
    kind_list = kind_list.lower().split(' ')
    
    n = len(kind_list)
    if n == 0:
        print('error: length of kind is 0, cannot loop')
        return 'error'
    
    #will try each element in the kind list and see if it is in species_XXX
    #if not, will keep looping over the length of n
    for i in range(n):
        if kind_list[i] in species_vft:
            return 'dionaea muscipula'
        elif kind_list[i] in species_neps:
            return 'nepenthes'
        elif kind_list[i] in species_sarracenia:
            return 'sarracenia'    
        elif kind_list[i] in species_sundew:
            return 'drosera'
        elif kind_list[i] in species_pings:
            return 'pinguicula'
        elif kind_list[i] in species_utric:
            return 'utricularia'
        elif kind_list[i] in species_cephs:
            return 'cephalotus'
        elif kind_list[i] in species_darlingtonia:
            return 'darlingtonia'
        elif kind_list[i] in species_heli:
            return 'heliamphora'
        elif i == (n-1):
            #when list of strings are not found in species_XXX, return other
            return 'other'

In [20]:
[len(names),len(prices),len(saleprices),len(soldouts),len(stores),len(species)]

[0, 0, 0, 0, 0, 0]

In [21]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Pearl River Exotics
URL = 'https://www.pearlriverexotics.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('a',attrs={'class':'site-nav__link site-nav__link--main'})
links = links[:len(links)-1] #excluding last link "carnivorous plants for beginners'

for link in links:
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path

    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    #finds how many pages to loop over
    try:
        page_count = soup.find('li',attrs={'class':'pagination__text'}).text.strip()
        page_count = page_count[len(page_count)-1:] #grabs how many pages to loop through
        page_count = int(page_count)
    except (AttributeError,IndexError):
        page_count = 1 #only 1 page

    for page_number in range(page_count):
        #wait random time between 0-5 seconds before scraping data
        r = random.randint(0,5)
        time.sleep(r)
    
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('a',attrs={'class':'grid-view-item__link'})

        for container in containers:
            name = container.find('div',attrs={'class':'h4 grid-view-item__title'}).text.strip()
            specie = check_species(name)

            price = container.find('span',attrs={'class':'product-price__price'}).text.strip()
            price = price[1:] #remove the 1st character of the string '$'
    
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Pearl River Exotics']
            saleprices = saleprices + [''] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!change when on sale
            species = species + [specie]
            
            #looks for sold out items
            try:
                soldout = container.find('span',attrs={'class':'product-price__sold-out'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
        
        #once page is scraped, grabs the URL for next page
        try:
            path = soup.findAll('a',attrs={'class':'btn btn--secondary btn--narrow'})
            path = path[len(path)-1] #gets the link to the next page
            path = path['href']
            URL2 = URL + path
        except (AttributeError,IndexError):
            continue #nothing

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df = pd.DataFrame(d)
df

,Name,Price,Sale Price,Sold Out,Store,Species
0,72+ Hour Heat Pack - For shipping with plants,4.00,,,Pearl River Exotics,other
1,Spagmoss 100 gram briquette,7.00,,,Pearl River Exotics,other
2,"Nepenthes robcantleyi ""Queen of Hearts"" x ""Kin...",149.00,,,Pearl River Exotics,nepenthes
3,Nepenthes veitchii BE-3734 'Bario squat' (red-...,39.00,,sold out,Pearl River Exotics,nepenthes
4,Nepenthes aristolochioides x burkei BE-3683,16.00,,sold out,Pearl River Exotics,nepenthes
...,...,...,...,...,...,...
64,"Nepenthes robcantleyi ""Queen of Hearts"" x fusc...",24.00,,,Pearl River Exotics,nepenthes
65,Nepenthes villosa *COLLECTABLE HARD ENAMEL PIN*,8.00,,,Pearl River Exotics,nepenthes
66,Nepenthes veitchii *COLLECTABLE HARD ENAMEL PIN*,8.00,,,Pearl River Exotics,nepenthes
67,Nepenthes edwardsiana *COLLECTABLE ENAMEL PIN*,8.00,,,Pearl River Exotics,nepenthes


In [22]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#California Carnivores
URL = 'https://www.californiacarnivores.com/'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
soup = soup.find('ul',attrs={'class':'sidebar-module__list'})
links = soup.findAll('a')
links = [links[i] for i in range(len(links)) if i not in (0,1,3,13,16,22,26,30,31,32,33,34,35,36,37)]
#exluding following pages: plant collections (0), easy to grow (1), drosera (3), sarracenia (13),
#nepenthes (16), pinguicula (22), utricularia (26), waterwheel, ..., gifts (30-37)

for link in links:
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page_next = True
    while page_next:
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half'})
        containers_sale = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half on-sale'})

        #grabs information of normal items
        for container in containers:
            name = container.find('p',attrs={'class':'grid-link__title'}).text.strip()
            specie = check_species(name)

            price = container.find('p',attrs={'class':'grid-link__meta'}).text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+2:] #remove the all characters before and including the string '$ '
    
            names = names + [name]
            prices = prices + [price]
            saleprices = saleprices + ['']
            stores = stores + ['California Carnivores']
            soldouts = soldouts + ['']
            species = species + [specie]
        
        #grabs information of sale items
        for container in containers_sale:
            name = container.find('p',attrs={'class':'grid-link__title'}).text.strip()
            specie = check_species(name)

            price = container.find('p',attrs={'class':'grid-link__meta'}).text.strip() #price of items on sale should be of form '$ xx.xx\n$ xx.xx'
            separate_index = price.rfind('$') #from end of string, look for 1st instance of $
            price_before = price[separate_index:] #gets original price
            price_before = price_before[2:] #remove the first 2 characters of the string '$ '
            price_after = price[:separate_index] #gets sale price
            price_after = price_after[2:] #remove the first 2 characters of the string '$ '
            price_after = price_after[:-1] #remove the last 2 characters of the string '$ '
    
            names = names + [name]
            prices = prices + [price_before]
            saleprices = saleprices + [price_after]
            stores = stores + ['California Carnivores']
            soldouts = soldouts + ['']
            species = species + [specie]
        
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            path = soup.find('a',attrs={'title':'Next »'})
            path = path['href']
            URL2 = URL + path
        except TypeError:
            page_next = False
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1

,Name,Price,Sale Price,Sold Out,Store,Species
0,Carnivorous Plant Conservation Donation,1.00,,,California Carnivores,other
1,Dionaea m. 'Alien' Potted,39.99,,,California Carnivores,dionaea muscipula
2,Dionaea m. 'B-52' Potted,14.99,,,California Carnivores,dionaea muscipula
3,Dionaea m. 'Brutal Shark' Potted,29.99,,,California Carnivores,dionaea muscipula
4,Dionaea m. 'Burbank's Best' Potted,14.99,,,California Carnivores,dionaea muscipula
...,...,...,...,...,...,...
127,72 Hour Heat Pack,3.99,,,California Carnivores,other
128,Live Sphagnum Moss,19.99,,,California Carnivores,other
129,Utricularia fulva Potted,14.99,,,California Carnivores,utricularia
130,72 Hour Heat Pack,3.99,,,California Carnivores,other


In [23]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Cook's Carnivorous Plants
URL = 'http://www.flytraps.com/Scripts/'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('span',attrs={'class':'CPcatDescProd'})
links = links[:22] #from cobra lily to nepenthes hybrid lowland

for link in links:
    #Checks what type of species
    kind = link.text
    specie = check_species(kind)
        
    #Create URL2 to be URL of species type
    path = link.a['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('div',attrs={'class':'prod-classic'})
    containers_count = len(containers)
    
    #print(URL2)
    #If is not blank, scrapes info. otherwise skips page
    if containers_count != 0:
        
        #each page contains 15 items, goes to next page when page_next mod 15 == 0
        page_next = 0
        while page_next % 15 == 0:
            page = urlopen(URL2)
            soup = BeautifulSoup(page,'html.parser')
            containers = soup.findAll('div',attrs={'class':'prod-classic'})
    
            for container in containers:
                name = container.h3.text.strip()
            
                #Most items on site are on sale. Try to process items as if the were on sale. If not on sale, then exception
                try:
                    price = container.find('del',attrs={'class':'CPprodLPriceV'}).text.strip()
                    price = price[1:] #removed '$' at beginning
            
                    saleprice = container.find('span',attrs={'class':'price'}).text.strip()
                    separate_index = saleprice.find('$') #look for 1st instance of $
                    saleprice = saleprice[separate_index+1:] #remove the all characters before and including the string '$'
                    saleprice = saleprice[:-3] #remove ' ea' from end of sales price
                except AttributeError:
                    price = container.find('span',attrs={'class':'price'}).text.strip()
                    separate_index = price.find('$') #look for 1st instance of $
                    price = price[separate_index+1:] #remove the all characters before and including the string '$'
                    price = price[:-3] #remove ' ea' from end of sales price
                
                    saleprice = ''
            
                names = names + [name]
                prices = prices + [price]
                stores = stores + ['Cook\'s Carnivorous Plants']
                soldouts = soldouts + ['']
                species = species + [specie]
                saleprices = saleprices + [saleprice]
                page_next = page_next + 1
                
            #Updates the URL for next page; stops the while loop if there is no next page
            if page_next % 15 == 0:
                paths = soup.find('div',attrs={'class':'spacing fl-right'})
                paths = paths.findAll('a')
                path = paths[len(paths)-1]['href'] #href of the last link (should be next page)
                URL2 = URL + path
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1

,Name,Price,Sale Price,Sold Out,Store,Species
0,Darlingtonia Californica Cobra Lily unpotted s...,17.00,12.00,,Cook's Carnivorous Plants,darlingtonia
1,Darlingtonia Californica Cobra Lily unpotted n...,28.00,20.00,,Cook's Carnivorous Plants,darlingtonia
2,drosera scorpiodes unpotted near flowering siz...,5.00,3.25,,Cook's Carnivorous Plants,drosera
3,Cephalotus follicularis Charles Brewer potted ...,45.00,25.00,,Cook's Carnivorous Plants,cephalotus
4,venus flytrap bigmouth unpotted 2-3 year old s...,10.00,7.00,,Cook's Carnivorous Plants,dionaea muscipula
...,...,...,...,...,...,...
100,Nepenthes burbidgeae seed grown unpotted 4+ in...,60.00,40.00,,Cook's Carnivorous Plants,nepenthes
101,Nepenthes klossii unpotted 2-3 inch leafspan p...,400.00,275.00,,Cook's Carnivorous Plants,nepenthes
102,Nepenthes klossii unpotted 4+ inch leafspan plant,500.00,375.00,,Cook's Carnivorous Plants,nepenthes
103,Nepenthes spectabilis large form seed grown un...,45.00,35.00,,Cook's Carnivorous Plants,nepenthes


In [24]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Carnivero
URL = 'https://www.carnivero.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('div',attrs={'class':'grid__item small--one-half medium-up--one-fifth slide-up-animation animated'})
links = [links[i] for i in range(10) if i!=1]  #all categories except beginner plants

#Looks at plant categories page
for link in links:            
    #Create URL2 to be URL of species type
    path = link.a['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
  
    page_next = True
    while page_next:
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('div',attrs={'class':'product grid__item medium-up--one-third small--one-half slide-up-animation animated'})
    
        for container in containers:
            name = container.find('div',attrs={'class':'product__title text-center'}).text.strip()
            specie = check_species(name)

            try:
                price = container.find('span',attrs={'class':'product__price'}).text.strip()
                separate_index = price.find('$') #look for 1st instance of $
                price = price[separate_index+1:] #remove the all characters before and including the string '$'
                prices = prices + [price]
                saleprices = saleprices + ['']           
            except AttributeError:
                #grab original price
                price = container.find('s').text.strip()
                separate_index = price.find('$') #look for 1st instance of $
                price = price[separate_index+1:] #remove the all characters before and including the string '$'
                prices = prices + [price]
            
                #grab new sales price
                price = container.find('span',attrs={'class':'product__price--on-sale'}).text.strip()
                separate_index = price.find('$') #look for 1st instance of $
                price = price[separate_index+1:] #remove the all characters before and including the string '$'
                saleprices = saleprices + [price] 
            
            try:
                soldout = container.find('strong',attrs={'class':'sold-out-text'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
            
            names = names + [name]
            stores = stores + ['Carnivero']
            species = species + [specie]
        
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            path = soup.find('span',attrs={'class':'next'})
            path = path.a['href']
            URL2 = 'https://www.carnivero.com' + path
        except AttributeError:
            page_next = False
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1

,Name,Price,Sale Price,Sold Out,Store,Species
0,Nepenthes veitchii H/L x burbidgeae,149.99,,,Carnivero,nepenthes
1,Nepenthes ventricosa x ovata - DM033,49.99,,,Carnivero,nepenthes
2,Nepenthes pectinata x ventricosa,39.99,,sold out,Carnivero,nepenthes
3,"Nepenthes 'Splendid Diana' x ventricosa red, C...",24.99,,sold out,Carnivero,nepenthes
4,"Nepenthes ampullaria x veitchii, BE-3720",29.99,,sold out,Carnivero,nepenthes
...,...,...,...,...,...,...
386,Drosera fulva,29.99,,,Carnivero,drosera
387,Drosera darwinensis,29.99,,,Carnivero,drosera
388,Drosera aff. dilato-petiolaris,29.99,,,Carnivero,drosera
389,Drosera broomensis,29.99,,,Carnivero,drosera


In [25]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Native Exotics
URL = 'https://nativeexoticsonline.com'
h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(URL,headers=h) 
page = urlopen(req).read() 
soup = BeautifulSoup(page,'html.parser')
links = soup.find('ul',attrs={'class':'dropdown'})
links = links.findAll('a')
pages_of_interest = [5] + list(range(10,15)) #All Neps + Cephs,Drosera,..., Utrics
links = [links[i] for i in pages_of_interest]

#Looks at plant categories page
for link in links:      
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page_next = True
    while page_next:
        req = Request(URL2,headers=h) 
        page = urlopen(req).read() 
        soup = BeautifulSoup(page,'html.parser')
        container_big = soup.find('ul',attrs={'class':'products columns-4'})
        containers = container_big.findAll('li')
    
        for container in containers:
            name = container.find('h2').text.strip()
            specie = check_species(name)
        
            price_list = container.findAll('span',attrs={'class':'woocommerce-Price-amount amount'})
            #2 prices listed when on sale. 1st list element is original price, 2nd element is sales price
            if len(price_list) == 2:
                price = price_list[0].text.strip()
                price = price[2:]
                saleprice = price_list[1].text.strip()
                saleprice = saleprice[2:]             
            elif len(price_list) == 1:
                price = price_list[0].text.strip()
                price = price[2:]
                saleprice = ''
        
            try:
                soldout = container.find('span',attrs={'class':'soldout'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
            
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Native Exotics']
            saleprices = saleprices + [saleprice]
            species = species + [specie]
            
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            URL2 = soup.find('a',attrs={'class':'next page-numbers'})
            URL2 = URL2['href']
        except TypeError:
            page_next = False
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1

In [26]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Carnivorous Plant Nursery
URL = 'https://carnivorousplantnursery.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('article')
links = links[:9] #Only looks at VFT to buterworts

#Looks at plant categories page
for link in links:      
    #Create URL2 to be URL of species type
    path = link.find('figure')
    path = path.a['href']
    URL2 = URL + path   
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('article',attrs={'class':'productgrid--item imagestyle--cropped-small '})
    for container in containers:
        name = container.find('h2').text.strip()
        specie = check_species(name)
    
        price = container.find('div',attrs={'class':'price--main'}).text.strip()
        separate_index = price.find('$') #look for 1st instance of $
        price = price[separate_index+1:] #remove the all characters before and including the string '$'
    
        names = names + [name]
        prices = prices + [price]
        stores = stores + ['Carnivorous Plant Nursery']
        saleprices = saleprices + [''] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!change when on sale
        species = species + [specie]
        
        #looks for sold out items
        try:
            soldout = container.find('span',attrs={'class':'productitem--badge badge--soldout'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True) 
df1

In [27]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Bergen Water Gardens
#cannot extract next page button functionality; cannot extract how many pages total
#instead keeps going to next page until error
try:
    page_number = 1
    while page_number < 11:
        URL = 'https://bergenwatergardens.com/product-category/carnivorous-plants/?fwp_paged=%d' % page_number
        h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        req = Request(URL,headers=h) 
        page = urlopen(req).read() 
        soup = BeautifulSoup(page,'html.parser')
    
        #wait random time between 0-5 seconds before scraping data
        r = random.randint(0,5)
        time.sleep(r)
        
        containers = soup.find('ul',attrs={'class':'products columns-4'})
        containers = containers.findAll('li')        
        
        for container in containers:
            name = container.find('h2').text.strip()
            specie = check_species(name)

            price = container.find('span',attrs={'class':'price'}).text.strip()
            #if item is on sale, gets normal price and sale price
            try:
                container.find('span',attrs={'class':'onsale'}).text.strip() == 'Sale!'
    
                separate_index = price.find('$') #from BEGINNING of string, look for 1st instance of $
                price_before = price[separate_index:] #gets original price
                price_before = price_before[1:7] #remove everything except price
        
                separate_index = price.rfind('$') #from END of string, look for 1st instance of $
                price_after = price[separate_index:] #gets original price
                price_after = price_after[1:7] #remove everything except price
    
                price = price_before
                saleprice = price_after
            #else just get the normal price
            except AttributeError:
                price = container.find('span',attrs={'class':'price'}).text.strip()
                price = price[2:] #removes '$ ' before price
                saleprice = ''
    
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Bergen Water Gardens']
            saleprices = saleprices + [saleprice]
            species = species + [specie]        
        
            #looks for sold out items
            try:
                URL2 = container.a['href']
                req = Request(URL2,headers=h) 
                page = urlopen(req).read() 
                soup = BeautifulSoup(page,'html.parser')
                
                soldout = soup.find('p',attrs={'class':'stock out-of-stock'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
                        
        #increase page_number for while loop
        page_number = page_number + 1
#if container is empty, pass- does nothing, 'pass' in python instead of 'continue'
except (AttributeError,IndexError):
    pass 
    

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True) 
df1

,Name,Price,Sale Price,Sold Out,Store,Species
0,72 Hour Heat Pack,3.00,,,Bergen Water Gardens,other
1,Akai Ryu Venus Fly Trap,12.00,,,Bergen Water Gardens,dionaea muscipula
2,Bugout™,35.00,,,Bergen Water Gardens,other
3,Cape Sundew Drosera capensis,12.00,,out of stock,Bergen Water Gardens,drosera
4,Fraser Island Sundew Drosera spatulata,8.00,,out of stock,Bergen Water Gardens,drosera
...,...,...,...,...,...,...
155,Nepenthes singalana x (burbidgeae x edwardsian...,40.00,,,Bergen Water Gardens,nepenthes
156,Nepenthes singalana x burkei BE 3878,20.00,,,Bergen Water Gardens,nepenthes
157,Nepenthes singalana x hamata BE3955,50.00,,,Bergen Water Gardens,nepenthes
158,Nepenthes singalana x mirabilis var. globosa B...,20.00,,,Bergen Water Gardens,nepenthes


In [28]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Petflytrap
URL = 'https://www.petflytrap.com/'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
soup = soup.find('table')
links = soup.findAll('td',attrs={'style':'text-indent:45px;'})
links = [links[i] for i in range(len(links)) if i not in (0,3,10,11,12)]
#exluding following pages: All carnivorous plants (0), tropical pitcher plants (3)
#which is a duplicate of nepenthes (4), terrariums & kits (10-12)s

for link in links:
    #Create URL2 to be URL of species type
    path = link.a['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page_next = True
    while page_next:
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = containers = soup.findAll('td',attrs={'width':'20%'})
        
        #if there are no items on the page, skips page
        if len(containers) == 0:
            break
        #else grabs information of normal items
        else:
            for container in containers:
                #try processing container info
                try:
                    name = container.text
                    separate_index = name.rfind('Y') #finds the index for 'Y' to remove everything after 'Your Price...'
                    separate_index
                    name = name[:separate_index] #removes all text after 'Your Price'
                    name = name[6:-4] #removes all newlines before and after name
                    specie = check_species(name)

                    price = container.find('td',attrs={'class':'price-info'}).text.strip()
                    #if item is on sale, seperate into price_before and price_after
                    if 'On sale' in price:
                        separate_index = price.find('$') #from BEGINNING of string, look for 1st instance of $
                        price_before = price[separate_index:] #gets original price
                        price_before = price_before[1:6] #remove everything except price
        
                        separate_index = price.rfind('$') #from END of string, look for 1st instance of $
                        price_after = price[separate_index:] #gets original price
                        price_after = price_after[1:6] #remove everything except price
                    else:
                        separate_index = price.rfind('$') #from end of string, look for 1st instance of $
                        price_before = price[separate_index:] #gets original price
                        price_before = price_before[1:] #remove the first 2 characters of the string '$ '
                        price_after = ''
    
                    names = names + [name]
                    prices = prices + [price_before]
                    saleprices = saleprices + [price_after]
                    stores = stores + ['Petflytap']
                    soldouts = soldouts + ['']
                    species = species + [specie]
                #if container is empty, pass- does nothing, 'pass' in python instead of 'continue'
                except AttributeError:
                    pass
        
            #Updates the URL for next page; stops the while loop if there is no next page
            try:
                path = soup.findAll('font',attrs={'size':'1'})
                path = path[len(path)-1].a['href']
                URL2 = URL + path
            except (TypeError,IndexError):
                page_next = False
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df1 = pd.DataFrame(d)
df = df.append(df1,ignore_index=True)
df1

HTTPError: HTTP Error 403: Forbidden

In [29]:
#Export concatinated df to Excel
df.to_excel("df.xlsx")

In [30]:
#Converting exported df back into correct format
df = pd.read_excel('df_2020_02_05.xlsx')
df = df.drop(columns='Unnamed: 0') #removing column of indecies

df

,Name,Price,Sale Price,Sold Out,Store,Species
0,72+ Hour Heat Pack - For shipping with plants,4.00,NaN,NaN,Pearl River Exotics,other
1,Spagmoss 100 gram briquette,7.00,NaN,NaN,Pearl River Exotics,other
2,"Nepenthes robcantleyi ""Queen of Hearts"" x ""Kin...",149.00,NaN,NaN,Pearl River Exotics,nepenthes
3,Nepenthes veitchii BE-3734 'Bario squat' (red-...,39.00,NaN,sold out,Pearl River Exotics,nepenthes
4,Nepenthes aristolochioides x burkei BE-3683,16.00,NaN,sold out,Pearl River Exotics,nepenthes
...,...,...,...,...,...,...
2131,Nepenthes singalana x (burbidgeae x edwardsian...,40.00,NaN,NaN,Bergen Water Gardens,nepenthes
2132,Nepenthes singalana x burkei BE 3878,20.00,NaN,NaN,Bergen Water Gardens,nepenthes
2133,Nepenthes singalana x hamata BE3955,50.00,NaN,NaN,Bergen Water Gardens,nepenthes
2134,Nepenthes singalana x mirabilis var. globosa B...,20.00,NaN,NaN,Bergen Water Gardens,nepenthes


In [ ]:
#Once the complete database is compiled, apply tagging function - tags any species found in product
#https://www.carnivorousplants.co.uk/resources/nepenthes-interactive-guide/
#https://en.wikipedia.org/wiki/List_of_Nepenthes_species

#compare product to database of species known
#if str in database, then species = str

In [ ]:
n = 172
name_nepenthes = [''] * n
URL = 'https://en.wikipedia.org/wiki/List_of_Nepenthes_species'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('tr')

for i in range(1,n):
    container = containers[i]
    name = container.find('a')
    name_nepenthes[i] = name['title'][10:] #removing 'Nepenthes ' from string
    
name_nepenthes = name_nepenthes[1:] #remove header row

In [ ]:
n = 1
name_nepenthes = [''] * n
URL = 'https://en.wikipedia.org/wiki/List_of_Nepenthes_species'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('tr')

for i in range(1,n):
    container = containers[i]
    name = container.find('a')
    name_nepenthes[i] = name['title'][10:] #removing 'Nepenthes ' from string
    
name_nepenthes = name_nepenthes[1:] #remove header row

In [ ]:
df['Cross1'] = ''
df['Cross2'] = ''
df['Cross3'] = ''
df['Cross4'] = ''
df['Cross5'] = ''
df['Cross6'] = ''

#i is row index, j is column index
#len(df) gives number of rows in df
for i in range(len(df)):
    string_split = df.at[i,'Name'].replace('(','').replace(')','').replace('[','').replace(']','').replace(',','')
    #removing all of the special characters:()[] so the string is split into 'ventricosa' and not '(ventricosa''
    string_split = string_split.split(' ')
    count = 1                           #counter for column name
    for j in range(len(string_split)):
        if string_split[j] in name_nepenthes:
            column = 'Cross%d' % count
            count = count + 1
            df.loc[[i],[column]] = string_split[j]

            
            

#also consider the cases for natural/common hybrids: ventrata, miranda
#create special if clause when tag is not found, open product page and scan description text for tags
df

In [ ]:
#Check what does not have any Cross1 tag
for i in range(len(df)):
    if df.at[i,'Cross1'] is '':
        print(df.at[i,'Name'])